<a href="https://colab.research.google.com/github/auralmn/aura_liquidmoe_snn_llama-3.2-3B_Notebooks/blob/main/Aura_LiquidMoe_Llama3_2_SNN_HEBBIAN_OJA_SANGER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AURA LIQUID MOE (AKA TEST 4.0 - AMYGDALA EXPERTS)

# AURA LIQUIDMOE SNN + LLAMA 3.2 3B UNSLOTH LLM (L4-T4)


## install dependencies

In [ ]:
# Install required libraries
!pip uninstall wandb
!pip install transformers torch sentence-transformers accelerate numpy
!pip install huggingface_hub
# 1. Install Unsloth
!pip install "unsloth[colab-new]"

!pip install huggingface_hub datasets


# 3. Authenticate (from Step 13)
from huggingface_hub import login
import os
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("Hugging Face login successful!")
except ImportError:
    print("Not in Colab or 'HF_TOKEN' secret not found.")
# --- Log into Hugging Face ---
# This is the best way to do it in Colab.
# 1. Go to your Colab notebook.
# 2. Click the "Key" icon (Secrets) in the left-hand sidebar.
# 3. Create a new secret named "HF_TOKEN".
# 4. Paste your Hugging Face Pro token (it starts with "hf_") as the value.
# 5. Make sure "Notebook access" is toggled ON.

# This code will then automatically find and use your secret.
from huggingface_hub import login
import os

try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("Hugging Face login successful!")
except ImportError:
    print("Not in Colab or 'HF_TOKEN' secret not found.")
    # You can paste your token manually if prompted
    # login()

## SETUP CLEAN AURA INSTANCE

In [ ]:
import enum

In [ ]:
# --- All Imports ---
from enum import Enum
import random
import uuid
import enum
import numpy as np
import random
import json
import itertools
import torch
from transformers import AutoTokenizer, logging
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
import os
from unsloth import FastLanguageModel
from typing import List, Dict, Optional, Union, Any, Tuple
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from numpy import ndarray
import asyncio
import io
import csv
import threading # For Memory Pool
import time
from datasets import load_dataset # <-- NEW: For GoEmotions

# --- PyTorch Imports (for Pre-Training) ---
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Suppress transformer warnings
logging.set_verbosity_error()

# --- 0. Colab/HF Login ---
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN, add_to_git_credential=False)
    print("Hugging Face login successful!")
except (ImportError, KeyError):
    print("HF_TOKEN secret not found or not in Colab. Ensure you are logged in.")

# --- 1. Memory Pool (from memory_pool.py) ---
@dataclass
class PoolStats:
    hits: int = 0; misses: int = 0; total_allocations: int = 0; peak_usage_mb: float = 0.0
class ArrayPool:
    """High-performance array pool"""
    def __init__(self, max_pool_size_mb: int = 512):
        self.max_pool_size = max_pool_size_mb * 1024 * 1024
        self.pools: Dict[Tuple[tuple, np.dtype], List[np.ndarray]] = {}
        self.current_usage = 0; self.stats = PoolStats(); self._lock = threading.Lock()
    def get_array(self, shape: tuple, dtype: np.dtype = np.float32, zero_fill: bool = True) -> np.ndarray:
        key = (shape, dtype)
        with self._lock:
            if key in self.pools and self.pools[key]:
                arr = self.pools[key].pop(); self.stats.hits += 1
                if zero_fill: arr.fill(0)
                return arr
            else:
                arr = np.empty(shape, dtype=dtype);
                if zero_fill: arr.fill(0)
                self.stats.misses += 1; self.stats.total_allocations += 1
                return arr
    def return_array(self, arr: np.ndarray) -> None:
        if arr is None: return
        key = (arr.shape, arr.dtype); array_size = arr.nbytes
        with self._lock:
            if self.current_usage + array_size <= self.max_pool_size:
                if key not in self.pools: self.pools[key] = []
                arr.fill(0); self.pools[key].append(arr); self.current_usage += array_size
                self.stats.peak_usage_mb = max(self.stats.peak_usage_mb, self.current_usage / (1024 * 1024))
_global_pool = ArrayPool()
def get_pooled_array(shape: tuple, dtype: np.dtype = np.float32, zero_fill: bool = True) -> np.ndarray:
    return _global_pool.get_array(shape, dtype, zero_fill)
def return_pooled_array(arr: np.ndarray) -> None:
    _global_pool.return_array(arr)

# --- 2. Optimized Whitener (from training_coordinator_optimized.py) ---
class OptimizedWhitener:
    """Memory-efficient online whitener"""
    def __init__(self, dim: int, eps: float = 1e-6, momentum: float = 0.01):
        self.dim = dim; self.eps = np.float32(eps); self.momentum = np.float32(momentum)
        self.mu = np.zeros(dim, dtype=np.float32); self.var = np.ones(dim, dtype=np.float32)
    def transform(self, x: np.ndarray) -> np.ndarray:
        if x.dtype != np.float32: x = x.astype(np.float32)
        _temp_diff = get_pooled_array((self.dim,), dtype=np.float32)
        _temp_result = get_pooled_array((self.dim,), dtype=np.float32)
        self.mu *= (1.0 - self.momentum); self.mu += self.momentum * x
        np.subtract(x, self.mu, out=_temp_diff)
        np.multiply(_temp_diff, _temp_diff, out=_temp_result)
        self.var *= (1.0 - self.momentum); self.var += self.momentum * _temp_result
        np.sqrt(self.var + self.eps, out=_temp_result)
        np.divide(_temp_diff, _temp_result, out=_temp_result)
        result_copy = _temp_result.copy()
        return_pooled_array(_temp_diff); return_pooled_array(_temp_result)
        return result_copy
    def state_dict(self) -> Dict: return {"mu": self.mu, "var": self.var}
    def load_state_dict(self, state: Dict): self.mu = state["mu"]; self.var = state["var"]

# --- 3. GoEmotions Labels (The Curriculum) ---
GOEMOTIONS_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
    'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

# --- 4. "Ears": FeatureGenerator (GoEmotions-Aware) ---
class FeatureGenerator:
    """Builds the 419-dim feature vector"""
    def __init__(self, sbert_model_name: str = "all-MiniLM-L6-v2"):
        self.SBERT_DIM = 384
        self.SINE_LENGTH = 32
        self.EXTRA_FEATURES = 3
        self.TOTAL_FEATURES = self.SBERT_DIM + self.SINE_LENGTH + self.EXTRA_FEATURES # 419
        print(f"CREATED: FeatureGenerator (Aura 6.0), Features: {self.TOTAL_FEATURES}")
        self.sbert_model = SentenceTransformer(sbert_model_name, device="cuda" if torch.cuda.is_available() else "cpu")
        # Educate the feature generator on the *real* emotion labels
        self.label_params = self._generate_default_params(GOEMOTIONS_LABELS)
        self.tokenizer = self.sbert_model.tokenizer
        self.vocab_size = self.tokenizer.vocab_size
        self.whitener = OptimizedWhitener(dim=self.TOTAL_FEATURES)

    def _generate_default_params(self, labels: List[str]) -> Dict[str, Dict]:
        base_freq = 1.5; base_phase = 0.5; params = {}
        for idx, label in enumerate(labels):
            params[label] = {"freq": base_freq + 0.1 * idx, "amp": 0.7, "phase": base_phase + 0.2 * idx}
        return params

    def build_features(self, record: Dict[str, Any], sbert_vec: np.ndarray) -> np.ndarray:
        prim = record.get("plutchik", {}).get("primary", "neutral")
        if isinstance(prim, list): prim = prim[0] if prim else "neutral"
        inten = float(record.get("plutchik", {}).get("intensity", 1.0))
        cfg = self.label_params.get(prim, {"freq": 1.0, "amp": 0.0, "phase": 0.0}) # Default to flat line
        t = np.linspace(0, 2*np.pi, self.SINE_LENGTH, dtype=np.float32)
        emb = (cfg["amp"] * inten * np.sin(cfg["freq"] * t + cfg["phase"])).astype(np.float32)
        text = record.get("text", "")
        extras = np.array([
            len(text) / 100.0,
            int("!" in text),
            int(record.get("tone", "none") in {"euphoric", "tense", "somber", "peaceful", "amazed"})
        ], dtype=np.float32)
        return np.concatenate([emb, extras, sbert_vec]).astype(np.float32)

    def generate_for_query(self, query: str) -> (np.ndarray, List[int]):
        record = {
            "text": query,
            "plutchik": {"primary": "neutral", "intensity": 0.5},
            "intent": "statement", "tone": "none"
        }
        sbert_vec = self.sbert_model.encode(query, normalize_embeddings=True)
        x_raw = self.build_features(record, sbert_vec)
        x_whitened = self.whitener.transform(x_raw)
        token_ids = self.tokenizer.encode(query, add_special_tokens=False)
        return x_whitened, token_ids

# --- 5. "Mouth": ResponseGenerator (Unsloth, Async) ---
class ResponseGenerator:
    def __init__(self, model_name: str = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"):
        print(f"CREATED: ResponseGenerator (Mouth) using Unsloth on '{model_name}'")
        max_seq_length = 2048; dtype = None; load_in_4bit = True
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name = model_name, max_seq_length = max_seq_length,
            dtype = dtype, load_in_4bit = load_in_4bit,
        )
        if self.tokenizer.pad_token is None: self.tokenizer.pad_token = self.tokenizer.eos_token
        print("   -> Unsloth Llama 3.1 model loaded successfully.")
    def _build_prompt(self, user_query: str, brain_state: dict) -> str:
        cns_state = brain_state.get('cns_state', 'ALERT').name
        stress = brain_state.get('stress_level', 0.0)
        persona = f"You are Aura, a bio-neural AI. Your current internal state is {cns_state}."
        if cns_state == 'HYPERVIGILANT' or stress > 1.0:
            persona += f" You are feeling a high-stress level ({stress:.2f}). Your response should be direct and acknowledge the tension."
        else: persona += " You are calm and helpful."
        messages = [{"role": "system", "content": persona}, {"role": "user", "content": user_query}]
        return self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    async def generate_response(self, user_query: str, brain_state: dict) -> str:
        print(f"\n--- 👄 ResponseGenerator (Unsloth) ---"); print(f"Generating response. Brain state: {brain_state}")
        prompt = self._build_prompt(user_query, brain_state)
        inputs = self.tokenizer([prompt], return_tensors="pt", padding=True, truncation=True, max_length=1024).to("cuda")
        terminators = [self.tokenizer.eos_token_id, self.tokenizer.convert_tokens_to_ids("<|eot_id|>")]
        outputs = await asyncio.to_thread(
            self.model.generate, **inputs, max_new_tokens=150, eos_token_id=terminators,
            do_sample=True, temperature=0.7, top_p=0.9,
        )
        response_text = self.tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[-1]:], skip_special_tokens=True)[0]
        print(f"Llama 3.1 (Unsloth) Output: {response_text}"); return response_text

# --- 6. SpikingAttention (k-WTA) (from snn_nlms_moe.py) ---
@dataclass
class SpikingAttention:
    """"""
    decay: float = 0.7; theta: float = 1.0; k_winners: int = 5
    gain_up: float = 1.5; gain_down: float = 0.6
    def compute_gains(self, token_seq: List[int], vocab_size: int) -> Optional[np.ndarray]:
        if not token_seq: return None
        v: Dict[int, float] = {}; spikes: Dict[int, int] = {}
        for j in token_seq:
            vj = self.decay * v.get(j, 0.0) + 1.0
            if vj >= self.theta: spikes[j] = spikes.get(j, 0) + 1; vj -= self.theta
            v[j] = vj
        ranked = sorted(spikes.items(), key=lambda kv: (-kv[1], -v.get(kv[0], 0.0)))
        winners = set(j for j,_ in ranked[:max(1, self.k_winners)])
        gains = np.ones(vocab_size, dtype=np.float64)
        seen = set(spikes.keys()) | set(v.keys())
        for j in seen:
            if 0 <= j < vocab_size: gains[j] = self.gain_up if j in winners else self.gain_down
        return gains

# --- 7. "Expert Neuron": NLMSHead (from snn_nlms_moe.py) ---
class NLMSHead:
    """This is the 'Expert'. It holds the weights and performs learning."""
    def __init__(self, n_features: int, n_outputs: int, vocab_size: int,
                 attention_config: Dict, mu: float = 0.1):
        self.n_features = n_features; self.n_outputs = n_outputs; self.vocab_size = vocab_size
        self.attention_config = attention_config; self.mu = mu; self._lock = asyncio.Lock()
        self.w = np.zeros((self.n_features, self.n_outputs), dtype=np.float64)
        self.b = np.zeros(self.n_outputs, dtype=np.float64)
        self.spiking_attention = SpikingAttention(**self.attention_config)
        self.last_error = np.zeros(self.n_outputs, dtype=np.float64)
        self.last_output = np.zeros(self.n_outputs, dtype=np.float64)
    def predict(self, x: np.ndarray) -> np.ndarray:
         x = np.asarray(x, dtype=np.float64).reshape(-1); return (x @ self.w) + self.b
    async def load_weights(self, w_path: str, b_path: str):
        """NEW: Load pre-trained weights from .npy files"""
        async with self._lock:
            try:
                self.w = np.load(w_path); self.b = np.load(b_path)
                print(f"   -> Successfully loaded weights: {w_path} (W:{self.w.shape}, b:{self.b.shape})")
            except Exception as e:
                print(f"   -> WARNING: Failed to load weights from {w_path}. Starting from zero. Error: {e}")
    async def step(self, x: np.ndarray, y_true: np.ndarray | float, token_ids: List[int]) -> np.ndarray:
        async with self._lock:
            x = np.asarray(x, dtype=np.float64).reshape(-1)
            y_hat = self.predict(x); self.last_output = y_hat
            y_true_vec = (np.array([y_true]) if np.isscalar(y_true)
                          else np.asarray(y_true, dtype=np.float64).reshape(-1))
            e = y_true_vec - y_hat; self.last_error = e
            attention_gains = self.spiking_attention.compute_gains(token_ids, self.vocab_size)
            avg_gain = 1.0
            if attention_gains is not None and token_ids:
                gains_for_seq = [attention_gains[token] for token in token_ids if 0 <= token < self.vocab_size]
                if gains_for_seq: avg_gain = np.mean(gains_for_seq)
            modulated_mu = self.mu * avg_gain
            x_norm_sq = 1e-8 + float(x @ x)
            grad = (x / x_norm_sq)[:, None] * e[None, :]
            self.w += modulated_mu * grad; self.b += modulated_mu * e
            return y_hat
    def state_dict(self) -> Dict: return {"w": self.w, "b": self.b}
    def load_state_dict(self, state: Dict): self.w = state["w"]; self.b = state["b"]

# --- 8. CNS & ThalamicRouter (Educated) ---
class ConsciousnessLevel(Enum):
    DEEP_SLEEP = 0; ASLEEP = 1; ALERT = 2; FOCUSED = 3; HYPERVIGILANT = 4
class CentralNervousSystem:
    """"""
    def __init__(self):
        self.consciousness_level = ConsciousnessLevel.ALERT
        self.stress_level = 0.0 # This is our 'cortisol'
        print("CREATED: CentralNervousSystem (CNS)")
    def set_consciousness(self, level: ConsciousnessLevel):
        if self.consciousness_level != level:
            self.consciousness_level = level; print(f"CNS: Consciousness set to {level.name}")
    def update_stress(self, error: float):
        new_stress = abs(error) * 1.5
        self.stress_level = (self.stress_level * 0.5) + (new_stress * 0.5)
        self.stress_level = max(0.0, self.stress_level - 0.1)
        if self.stress_level > 1.0:
            self.set_consciousness(ConsciousnessLevel.HYPERVIGILANT)
        else:
            self.set_consciousness(ConsciousnessLevel.ALERT)

class ThalamicRouter:
    """
    This router is now "educated" with the GoEmotions label map.
    It provides target signals (y_true) for the experts.
    """
    def __init__(self, label_maps: Dict):
        # --- THIS IS THE FIX ---
        self.label_maps = label_maps  # <-- ADD THIS LINE
        # --- END FIX ---

        self.emotion_map = label_maps.get('emotion', {})
        self.intent_map = label_maps.get('intent', {})
        print(f"CREATED: ThalamicRouter (Educated with {len(self.emotion_map)} emotions)")

    def get_target_signals(self, query: str) -> dict:
        query = query.lower()
        signals = {
            'emotion': self.emotion_map.get('neutral', 0.0),
            'intent': self.intent_map.get('statement', 0.0)
        }
        # Keyword-based mapping to the *real* GoEmotions labels
        if 'scared' in query or 'fear' in query or 'anxious' in query or 'nervous' in query:
            signals['emotion'] = self.emotion_map.get('fear', self.emotion_map.get('nervousness', 0.0))
        elif 'happy' in query or 'joy' in query or 'excited' in query:
            signals['emotion'] = self.emotion_map.get('joy', self.emotion_map.get('excitement', 0.0))
        elif 'angry' in query or 'annoyed' in query:
            signals['emotion'] = self.emotion_map.get('anger', self.emotion_map.get('annoyance', 0.0))
        elif 'sad' in query or 'grief' in query:
            signals['emotion'] = self.emotion_map.get('sadness', self.emotion_map.get('grief', 0.0))

        if '?' in query:
            signals['intent'] = self.intent_map.get('question', 0.0)
        elif '!' in query:
            signals['intent'] = self.intent_map.get('exclamation', 0.0)

        return signals

# --- 9. The Final IBNN (Aura 6.0 - Pre-Trained) ---
class IntegratedBioNeuralNetwork:
    """
    This is Aura 6.0. It learns from a "School" (offline training)
    and then fine-tunes in real-time (online learning).
    """
    def __init__(self, llm_model_name: str = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"):
        print("--- 🧠 Initializing Aura 6.0 (Pre-Trained) ---")
        self.feature_gen = FeatureGenerator()
        self.response_gen = ResponseGenerator(llm_model_name)
        self.cns = CentralNervousSystem()
        self.feature_dim = self.feature_gen.TOTAL_FEATURES
        self.vocab_size = self.feature_gen.vocab_size
        self.experts: Dict[str, NLMSHead] = {}
        self.router = None # Will be set after education
        self.education_dir = "/content/drive/MyDrive/aura_education_weights" # Save to Drive
        os.makedirs(self.education_dir, exist_ok=True)
        print("--- Aura 6.0 Brain Initialized (Awaiting Education) ---")

    def _add_expert(self, name: str, n_outputs: int, attention_config: dict):
        head = NLMSHead(
            n_features=self.feature_dim,
            n_outputs=n_outputs,
            vocab_size=self.vocab_size,
            attention_config=attention_config,
            mu=0.05
        )
        self.experts[name] = head
        print(f"Added Expert: {name} (Output dim: {n_outputs})")

    async def educate_brain(self):
        """
        Runs the 'School' pipeline to pre-train experts from GoEmotions.

        """
        print("\n--- 🎓 STARTING OFFLINE EDUCATION (GoEmotions) ---")

        # 1. Load GoEmotions dataset
        print("Loading 'goemotions' dataset from Hugging Face...")
        dataset = load_dataset("google-research-datasets/go_emotions", "raw")['train']
        # Let's use a subset for speed in Colab
        dataset = dataset.shuffle(seed=42).select(range(5000)) # 5000 samples

        all_texts = dataset['text']

        # 2. Precompute SBERT in a thread
        print(f"Pre-computing SBERT embeddings for {len(all_texts)} texts...")
        sbert_embeddings = await asyncio.to_thread(
            self.feature_gen.sbert_model.encode,
            all_texts, normalize_embeddings=True, batch_size=64
        )

        # 3. Create Label Maps
        emotion_labels = GOEMOTIONS_LABELS
        # Mock 'intent' labels as GoEmotions doesn't have them
        intent_labels = ["question", "statement", "exclamation", "request", "none"]
        label_maps = {
            'emotion': {label: idx for idx, label in enumerate(emotion_labels)},
            'intent': {label: idx for idx, label in enumerate(intent_labels)},
        }

        # 4. Build Features and Tensorize
        print("Building features and tensorizing...")
        X_features, y_emotion, y_intent = [], [], []

        for i, record in enumerate(dataset):
            # Create a mock 'plutchik' record from the multi-label data
            primary_emotion = 'neutral'
            for label in emotion_labels:
                if label != 'neutral' and record[label] == 1:
                    primary_emotion = label
                    break # Just take the first one

            mock_record = {
                "text": record['text'],
                "plutchik": {"primary": primary_emotion, "intensity": 1.0},
                "intent": "question" if "?" in record['text'] else "statement"
            }

            X_features.append(self.feature_gen.build_features(mock_record, sbert_embeddings[i]))
            y_emotion.append(label_maps['emotion'].get(primary_emotion, 27)) # 27 is 'neutral'
            y_intent.append(label_maps['intent'].get(mock_record['intent'], 1)) # 1 is 'statement'

        X_train = torch.tensor(np.stack(X_features), dtype=torch.float32)
        y_emotion_train = torch.tensor(y_emotion, dtype=torch.long)
        y_intent_train = torch.tensor(y_intent, dtype=torch.long)

        # --- 5. Train "Emotion" Expert ---
        print(f"Training Emotion expert ({self.feature_gen.TOTAL_FEATURES} -> {len(emotion_labels)} classes)...")
        emotion_model = LinearTorchModel(self.feature_gen.TOTAL_FEATURES, len(emotion_labels))
        optimizer = optim.AdamW(emotion_model.parameters(), lr=5e-3)
        criterion = nn.CrossEntropyLoss()
        for epoch in range(15): # 15 epochs
            optimizer.zero_grad(); loss = criterion(emotion_model(X_train), y_emotion_train)
            loss.backward(); optimizer.step()
        preds = emotion_model(X_train).argmax(dim=1); acc = (preds == y_emotion_train).float().mean().item()
        print(f"  -> Emotion training complete. Final Acc: {acc:.2f}")

        # --- 6. Train "Intent" Expert ---
        print(f"Training Intent expert ({self.feature_gen.TOTAL_FEATURES} -> {len(intent_labels)} classes)...")
        intent_model = LinearTorchModel(self.feature_gen.TOTAL_FEATURES, len(intent_labels))
        optimizer = optim.AdamW(intent_model.parameters(), lr=5e-3)
        criterion = nn.CrossEntropyLoss()
        for epoch in range(10): # 10 epochs
            optimizer.zero_grad(); loss = criterion(intent_model(X_train), y_intent_train)
            loss.backward(); optimizer.step()
        preds = intent_model(X_train).argmax(dim=1); acc = (preds == y_intent_train).float().mean().item()
        print(f"  -> Intent training complete. Final Acc: {acc:.2f}")

        # --- 7. Export Weights to Google Drive ---
        print(f"Exporting educated weights to {self.education_dir}...")
        export_linear_weights(emotion_model, self.education_dir, "emotion_classifier")
        export_linear_weights(intent_model, self.education_dir, "intent_classifier")

        # --- 8. "Mind Upload" - Load weights into NLMSHead Experts ---
        await self.load_education(label_maps, weights_dir=self.education_dir)

        print("--- 🎓 OFFLINE EDUCATION COMPLETE ---")

    async def load_education(self, label_maps: Dict, weights_dir: str = "."):
        """
        Create experts based on label maps and load pre-trained weights.

        """
        print(f"\n--- 📚 Loading Education from {weights_dir} ---")
        self.experts.clear() # Clear any old experts

        if 'emotion' in label_maps:
            self._add_expert(
                name='emotion',
                n_outputs=len(label_maps['emotion']),
                attention_config={'decay': 0.6, 'k_winners': 4, 'gain_up': 2.0, 'gain_down': 0.4}
            )
            await self.experts['emotion'].load_weights(
                w_path=os.path.join(weights_dir, "emotion_classifier_W.npy"),
                b_path=os.path.join(weights_dir, "emotion_classifier_b.npy")
            )
        if 'intent' in label_maps:
            self._add_expert(
                name='intent',
                n_outputs=len(label_maps['intent']),
                attention_config={'decay': 0.7, 'k_winners': 5, 'gain_up': 1.5, 'gain_down': 0.7}
            )
            await self.experts['intent'].load_weights(
                w_path=os.path.join(weights_dir, "intent_classifier_W.npy"),
                b_path=os.path.join(weights_dir, "intent_classifier_b.npy")
            )
        self.router = ThalamicRouter(label_maps)

    async def process_query(self, query: str) -> str:
        print(f"\n--- ☀️ PROCESSING QUERY (Aura 6.0): '{query}' ---")
        if not self.router:
            return "I am uneducated. Please run `await aura.educate_brain()`."

        target_signals = self.router.get_target_signals(query)
        cns_level = self.cns.consciousness_level
        x_whitened, token_ids = self.feature_gen.generate_for_query(query)

        tasks = []
        for name, expert in self.experts.items():
            num_classes = expert.n_outputs
            target_index = target_signals.get(name, 0)
            y_true = np.zeros(num_classes)
            y_true[target_index] = 1.0 # One-hot target
            tasks.append(expert.step(x_whitened, y_true, token_ids))
        await asyncio.gather(*tasks)

        emotion_error_vec = self.experts['emotion'].last_error
        stress = np.mean(np.abs(emotion_error_vec))
        self.cns.update_stress(stress)

        emotion_pred_index = self.experts['emotion'].last_output.argmax()
        emotion_pred_label = GOEMOTIONS_LABELS[emotion_pred_index]
        print(f"Emotion Prediction: {emotion_pred_label} (Error/Stress: {stress:.3f})")

        final_brain_state = {
            'cns_state': self.cns.consciousness_level,
            'stress_level': self.cns.stress_level,
        }
        response = await self.response_gen.generate_response(query, final_brain_state)
        self.last_run_final_stress = self.cns.stress_level
        return response

    async def save_brain(self, path: str):
        """Saves the *learned* (fine-tuned) brain state."""
        print(f"\n--- 💾 Saving brain state to {path} ---")
        state_dict = {}
        for name, expert in self.experts.items():
            async with expert._lock:
                state_dict[f"expert_{name}_w"] = expert.w
                state_dict[f"expert_{name}_b"] = expert.b
        state_dict["whitener_mu"] = self.feature_gen.whitener.mu
        state_dict["whitener_var"] = self.feature_gen.whitener.var
        await asyncio.to_thread(np.savez_compressed, path, **state_dict)
        print("--- 💾 Save complete ---")

    async def load_brain(self, path: str):
        """Loads a saved brain state from a .npz file."""
        print(f"\n--- 🧠 Loading brain state from {path} ---")
        try:
            data = np.load(path)
            for name, expert in self.experts.items():
                async with expert._lock:
                    if f"expert_{name}_w" in data:
                        expert.w = data[f"expert_{name}_w"]
                        expert.b = data[f"expert_{name}_b"]
                        print(f"   -> Loaded weights for expert: {name}")
            if "whitener_mu" in data:
                self.feature_gen.whitener.mu = data["whitener_mu"]
                self.feature_gen.whitener.var = data["whitener_var"]
            print("--- 🧠 Load complete ---")
        except Exception as e:
            print(f"--- ❌ ERROR: Failed to load brain state. {e} ---")

# --- 11. PyTorch Models (for the "School") ---
class LinearTorchModel(nn.Module):
    """"""
    def __init__(self, input_dim: int, num_classes: int):
        super().__init__(); self.fc = nn.Linear(input_dim, num_classes)
    def forward(self, x): return self.fc(x)

def export_linear_weights(model: LinearTorchModel, output_dir: str, task_name: str):
    """"""
    os.makedirs(output_dir, exist_ok=True)
    W = model.fc.weight.detach().cpu().numpy().T
    b = model.fc.bias.detach().cpu().numpy()
    np.save(os.path.join(output_dir, f"{task_name}_W.npy"), W)
    np.save(os.path.join(output_dir, f"{task_name}_b.npy"), b)
    print(f"Exported {task_name} weights: W{W.shape}, b{b.shape}")

In [ ]:
# --- This is the test cell for Aura 6.0 ---
# It must be run in a cell *after* Cell 50
import os
# --- 1. Mount Google Drive ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    SAVE_DIR = "/content/drive/MyDrive/aura_education_weights"
    print(f"Google Drive mounted. Will save/load brain from: {SAVE_DIR}")
except ImportError:
    print("Not in Colab. Saving to local directory './aura_brain_state_v6'")
    SAVE_DIR = "./aura_brain_state_v6"

os.makedirs(SAVE_DIR, exist_ok=True)
BRAIN_STATE_FILE = os.path.join(SAVE_DIR, "aura_6_memory.npz")
LABEL_MAPS_FILE = os.path.join(SAVE_DIR, "aura_6_labels.json")


# --- 2. The Main Async Test Loop ---
async def main_test_loop():

    # --- 3. INITIALIZE AND EDUCATE (Day 1) ---
    print("--- 🧠 Initializing Aura 6.0 (Loading LLMs with Unsloth...) ---")
    aura_day_1 = IntegratedBioNeuralNetwork(
        llm_model_name="meta-llama/Llama-3.2-3B-Instruct"
    )
    print("--- Aura LLM Components Loaded ---")

    # Run the "School"
    await aura_day_1.educate_brain()

    # Save the label maps
    label_maps = aura_day_1.router.label_maps
    with open(LABEL_MAPS_FILE, 'w') as f:
        json.dump(label_maps, f)

    # --- 4. TEST 1: (PRE-TRAINED, "scared") ---
    print("\n\n--- 🗣️ TEST 1: Educated Brain, 'Scared' Query ---")
    response_1 = await aura_day_1.process_query("I am feeling very scared")
    print(f"\nFINAL AURA RESPONSE (Day 1): {response_1}")
    day_1_stress = aura_day_1.last_run_final_stress

    # --- 5. TEST 2: (PRE-TRAINED, "happy") ---
    print("\n\n--- 🗣️ TEST 2: Educated Brain, 'Happy' Query ---")
    response_2 = await aura_day_1.process_query("I am so happy and full of joy!")
    print(f"\nFINAL AURA RESPONSE (Day 1, Happy): {response_2}")

    # --- 6. SAVE THE BRAIN ---
    # At this point, the brain has been fine-tuned on "scared" and "happy"
    await aura_day_1.save_brain(BRAIN_STATE_FILE)

    # --- 7. LOAD THE BRAIN (Simulate a restart) ---
    print("\n\n--- 🧠 Initializing NEW Aura 6.0 (Blank Brain) ---")
    aura_day_2 = IntegratedBioNeuralNetwork(
        llm_model_name="meta-llama/Llama-3.2-3B-Instruct"
    )

    # Give it the *same* education so its experts are the right shape
    print("--- Loading 'School' curriculum (label maps) ---")
    with open(LABEL_MAPS_FILE, 'r') as f:
        loaded_label_maps = json.load(f)

    # Load the *weights*
    await aura_day_2.load_education(label_maps=loaded_label_maps, weights_dir=SAVE_DIR)
    # Load the *fine-tuned state*
    await aura_day_2.load_brain(BRAIN_STATE_FILE)

    # --- 8. TEST 3: (LOADED BRAIN, "scared") ---
    print("\n\n--- 🗣️ TEST 3: LOADED Brain, 'Scared' Query (Day 2) ---")
    response_3 = await aura_day_2.process_query("I am feeling very scared")
    print(f"\nFINAL AURA RESPONSE (Day 2): {response_3}")
    day_2_stress = aura_day_2.last_run_final_stress

    # --- 9. VERIFICATION ---
    print("\n\n--- 🔬 PERSISTENCE VERIFICATION ---")
    print(f"Day 1 Final Stress (on 'scared'): {day_1_stress:.4f}")
    print(f"Day 2 Final Stress (on 'scared'): {day_2_stress:.4f}")

    if day_2_stress < (day_1_stress * 1.1): # Allow for small floating point differences
        print("✅ SUCCESS: The loaded brain's stress response was persistent. Learning was saved and restored!")
    else:
        print("❌ FAILURE: The loaded brain's stress was different. Persistence failed.")

# --- Run the async main loop ---
import nest_asyncio
nest_asyncio.apply()

asyncio.run(main_test_loop())